In [1]:
import pandas as pd
import numpy as np

# Spiking Features

In [ ]:
raw_spiking_df = pd.read_csv('Total_AP_time.csv')
spike_t = {}
for cell_id in raw_spiking_df.columns:
    spike_t[cell_id] = raw_spiking_df.loc[:, cell_id].dropna().values

In [ ]:
def get_spiking_features(spiking_t: np.ndarray, t_range=(0.2, 0.8), num_bins=30):
    hist, bin_edges = np.histogram(spiking_t, bins=num_bins, range=t_range)
    spiking_t_ = spiking_t.compress((spiking_t>=t_range[0])&(spiking_t<=t_range[1]))
    isi = np.diff(spiking_t_)
    adapt_index = (isi[1:]-isi[:-1])/(isi[1:]+isi[:-1])
    isi_cv = isi.std()/isi.mean()
    return {
        'spiking_t': spiking_t_,
        'spiking_hist': hist,
        'isi': isi,
        'adapt_index': adapt_index,
        'abs_AI': abs(adapt_index),
        'avg_adapt_index': adapt_index.mean(),
        'avg_abs_AI': abs(adapt_index).mean(),
        'isi_cv': isi_cv
    }

In [ ]:
demo_res=get_spiking_features(spike_t['20180903cell3']/1000)

In [ ]:
spiking_feature_df = pd.DataFrame({
    'cell_id': cell_id,
    **get_spiking_features(p/1000)
} for cell_id, p in spike_t.items())

In [ ]:
spiking_feature_df = spiking_feature_df.set_index('cell_id')
res_ai_isi_df = spiking_feature_df.select_dtypes(include=['float64'])
res_array_df = spiking_feature_df.select_dtypes(include=['object'])
with pd.ExcelWriter('spiking_features_0819.xlsx') as writer:
    res_ai_isi_df.to_excel(writer, sheet_name='AI_ISI')
    for col in res_array_df.columns:
        res_df = pd.DataFrame(res_array_df[col].tolist(), index=res_array_df.index)
        res_df.to_excel(writer, sheet_name=col)